In [1]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime
import time
from requests.exceptions import ConnectionError

In [2]:
def get_invoice_info(cookie,doc_id):
    url = f"https://ecosys.gov.vn/CertificatesUpgrade/Business/OrderDisplay.aspx?DocId={doc_id}"

    # Headers cần thiết
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Language": "en-US,en;q=0.9,vi;q=0.8,vi-VN;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": cookie,
        "DNT": "1",
        "Referer": "https://ecosys.gov.vn/CertificatesUpgrade/Business/CertificateAdvanceSearch.aspx",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36",
        "sec-ch-ua": '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
    }

    # Gửi yêu cầu GET
    response = requests.get(url, headers=headers)

    # Kiểm tra phản hồi
    if response.status_code == 200:
        print("Request thành công!")
        # print(response.text)  # In nội dung HTML của trang
    else:
        print(f"Lỗi: {response.status_code}")
        # print(response.text)

    invoice_info = {}
    invoice_info["doc_id"] = doc_id

    if response.text:
        soup = BeautifulSoup(response.text,"html.parser")
    info_soup = soup.find('div',id = "ctl00_cplhContainer_rpvRepresenter") 
    audit_info_soup = soup.find('div',id = "ctl00_cplhContainer_pnlAuditTrail") 


    try :
        tt = info_soup.find('span',id = "ctl00_cplhContainer_lblStatus").get_text().strip()
        invoice_info["status"] = tt
    except:
        print("not found status")
    try :
        ordercode = info_soup.find('span',id = "ctl00_cplhContainer_lblOrderCode").get_text().strip()
        invoice_info["ordercode"] = ordercode
    except:
        print("not found ordercode")
    try :
        companyname = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyName").get_text().strip()
        invoice_info["companyname"] = companyname
    except:
        print("not found companyname")
    try :
        companytaxcode = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyTaxcode").get_text().strip()
        invoice_info["companytaxcode"] = companytaxcode
    except:
        print("not found companytaxcode")
    try :
        companyaddress = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyAddress").get_text().strip()
        invoice_info["companyaddress"] = companyaddress
    except:
        print("not found companyaddress")
    try :
        companyemail = info_soup.find('span',id = "ctl00_cplhContainer_lblCompanyEmail").get_text().strip()
        invoice_info["companyemail"] = companyemail
    except:
        print("not found companyemail")
    try :
        amount = info_soup.find('span',id = "ctl00_cplhContainer_lblAmount").get_text().strip()
        invoice_info["amount"] = amount
    except:
        print("not found amount")
        
    try:
        co_table = info_soup.find('table',class_ = 'rgMasterTable')
        # Lấy tiêu đề
        co_table_header = [th.get_text(separator=" ", strip=True) for th in co_table.find('thead').find_all('th', class_ = "rgHeader")]

        # Lấy dữ liệu từng dòng trong bảng
        co_info_rows = []
        for tr in co_table.find('tbody').find_all('tr'):
            row = [td.get_text(separator=" ", strip=True) for td in tr.find_all('td')]
            co_info_rows.append(row)
    except:
        print("Not found goods_info")
    co_info = [co_table_header] + co_info_rows
    invoice_info["co_info"] = co_info
    
    try :
        invoice_code = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceCode").get_text().strip()
        invoice_info["invoice_code"] = invoice_code
    except:
        print("not found amoinvoice_codeunt")
    try :
        invoice_receip_no = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceReceiptNumber").get_text().strip()
        invoice_info["invoice_receip_no"] = invoice_receip_no
    except:
        print("not found invoice_receip_no")
    try :
        invoice_address_label = info_soup.find('span',id = "ctl00_cplhContainer_lblInvoiceAddressLabel")
        
        if invoice_address_label:
            try :
                a_element = invoice_address_label.find_next('a')  
                invoice_address_label = a_element['href'].strip()  
                # print(invoice_address_label)
            except:
                print("Thẻ <a> không tìm thấy hoặc không có thuộc tính href.") 
            
        invoice_info["invoice_address_label"] = invoice_address_label
    except:
        print("not found invoice_address_label")
    try :
        another_email = info_soup.find('span',id = "ctl00_cplhContainer_lblEmail01").get_text().strip()
        invoice_info["another_email"] = another_email
    except:
        print("not found another_email")    
        
    try :
        created_at = audit_info_soup.find('span',id = "ctl00_cplhContainer_AuditTrail1_lblCreatedAt").get_text().strip()
        invoice_info["created_at"] = created_at
    except:
        print("not found created_at")   
        
    try :
        last_modified_at = audit_info_soup.find('span',id = "ctl00_cplhContainer_AuditTrail1_lblLastModifiedAt").get_text().strip()
        invoice_info["last_modified_at"] = last_modified_at
    except:
        print("not found last_modified_at")   

    print(invoice_info)
    return invoice_info

In [3]:
with open("output_invoice_ids.txt", "r", encoding="utf-8") as file:
    invoice_list = [line.strip() for line in file]  

In [4]:
len(invoice_list)

258

In [5]:

cookie = 'Bifrost_SessionID=l14zt1mzkn0dg5bcgdxvsss1'

In [6]:


# Danh sách lưu thông tin hóa đơn
all_invoice_info = []

# Số lần thử lại tối đa
MAX_RETRIES = 10
RETRY_DELAY = 3  # Giây

for invoice in invoice_list:
    retries = 0
    success = False
    
    while retries < MAX_RETRIES and not success:
        try:
            invoice_info = get_invoice_info(cookie, invoice)
            all_invoice_info.append(invoice_info)
            success = True  # Xử lý thành công, thoát vòng lặp retry
        except ConnectionError as e:
            retries += 1
            print(f"Connection error for invoice {invoice}. Retry {retries}/{MAX_RETRIES}. Error: {e}")
            time.sleep(RETRY_DELAY)
        except Exception as e:
            retries += 1
            print(f"Error processing invoice {invoice}. Retry {retries}/{MAX_RETRIES}. Error: {e}")
            time.sleep(RETRY_DELAY)

    if not success:
        print(f"Failed to process invoice {invoice} after {MAX_RETRIES} retries.")

import pandas as pd
df = pd.DataFrame(all_invoice_info)
df.to_csv("all_invoice_info.csv",index=False, encoding='utf-8')

Request thành công!
{'doc_id': '1157556', 'status': 'Đã nộp phí', 'ordercode': '0001165F8', 'companyname': 'công ty tnhh Seojin Auto', 'companytaxcode': '2300956022', 'companyaddress': 'Lô J5, Khu công nghiệp Đại Đồng - Hoàn Sơn, Phường Tân Hồng, Thành phố Từ Sơn, Tỉnh Bắc Ninh, Việt Nam', 'companyemail': 'duongvietkhanh@seojinsys.co.kr', 'amount': '60,000 (VNĐ)', 'goods_info': [['Số C/O', 'Loại phí', 'Loại phí', 'Số tiền (VNĐ)'], ['VN-VN 24/01/064434', '1', 'C/O mới', '60,000']], 'invoice_code': '1133910', 'invoice_receip_no': '2300956022-0001165F8', 'invoice_address_label': 'https://0106196017.vnpt-invoice.com.vn', 'another_email': 'invoice@eximuni.com', 'created_at': '24/12/2024 13:33:20 PM', 'last_modified_at': '24/12/2024 13:40:30 PM'}
Request thành công!
{'doc_id': '1156617', 'status': 'Đã nộp phí', 'ordercode': '00011624C', 'companyname': 'công ty tnhh Seojin Auto', 'companytaxcode': '2300956022', 'companyaddress': 'Lô J5, Khu công nghiệp Đại Đồng - Hoàn Sơn, Phường Tân Hồng, Th